# Opening a New Restaurant in Indore, India
## Week 5 Final Report
* Build a dataframe of neighborhoods in Indore, India by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new Restaurant


In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

## **1. Importing Libraries**

In [2]:
import pandas as pd
import numpy as np
import json

from geopy.geocoders import Nominatim
import geocoder

import requests
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium
print("Libraries imported.")

Libraries imported.


## **2. Scrap data from Wikipedia page into a DataFrame**

In [3]:
data=requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Indore').text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
idf = pd.DataFrame({"Neighborhood": neighborhoodList})

idf

,Neighborhood
0,"Azad Nagar, Indore"
1,Bhanwar Kua
2,Bijalpur
3,"Chandan Nagar, Indore"
4,"Chandra Nagar, Indore"
5,Dewas Naka
6,Geeta Bhawan
7,Indore G.P.O.
8,Khandwa Naka
9,L.I.G. Colony


## **3. Get the geographical coordinates**

In [7]:
#import geocoder
def get_latlng(neighborhood):
    coords = None
    while(coords is None):
        g = geocoder.arcgis('{}, Indore, India'.format(neighborhood))
        coords = g.latlng
    return coords
    



In [8]:
coords = [ get_latlng(neighborhood) for neighborhood in idf["Neighborhood"].tolist() ]

In [9]:
coords

[[22.69658000000004, 75.88767000000007],
 [22.687640000000044, 75.86291000000006],
 [22.66324000000003, 75.84332000000006],
 [22.709380000000067, 75.82394000000005],
 [22.74147000000005, 75.89447000000007],
 [22.771764333333323, 75.90165866666665],
 [22.71822000000003, 75.88766000000004],
 [24.977950000000078, 77.85809000000006],
 [22.689258247414326, 75.87025921921465],
 [22.734780000000057, 75.88485000000003],
 [22.741695000000004, 75.89988900000002],
 [22.70479000000006, 75.87657000000007],
 [22.727690000000052, 75.88578000000007],
 [22.67416000000003, 75.84224000000006],
 [22.734780000000057, 75.88485000000003],
 [22.755351114631523, 75.89687741798681],
 [22.76022000000006, 75.87815000000006],
 [22.756410000000074, 75.89247000000006]]

In [10]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


In [11]:
idf['Latitude'] = df_coords['Latitude']
idf['Longitude'] = df_coords['Longitude']

In [12]:
print(idf.shape)
idf

(18, 3)


,Neighborhood,Latitude,Longitude
0,"Azad Nagar, Indore",22.696580,75.887670
1,Bhanwar Kua,22.687640,75.862910
2,Bijalpur,22.663240,75.843320
3,"Chandan Nagar, Indore",22.709380,75.823940
4,"Chandra Nagar, Indore",22.741470,75.894470
5,Dewas Naka,22.771764,75.901659
6,Geeta Bhawan,22.718220,75.887660
7,Indore G.P.O.,24.977950,77.858090
8,Khandwa Naka,22.689258,75.870259
9,L.I.G. Colony,22.734780,75.884850


In [13]:
# save the DataFrame as CSV file
idf.to_csv("idf.csv", index=False)

## **4. Map of Indore with neighborhoods superimposed on top**

In [14]:
#indore map
geolocator = Nominatim(user_agent="i_explorer")
city ="Indore"
country ="India"
loc = geolocator.geocode(city+','+ country)
latitude = loc.latitude
longitude = loc.longitude
print("latitude is :-" ,latitude,"\nlongtitude is:-" ,longitude)
map_indore = folium.Map([latitude, longitude], zoom_start=10)
map_indore

latitude is :- 22.7203616 
longtitude is:- 75.8681996


In [15]:
#adding markers to map
for lat, lng, neighborhood in zip(idf['Latitude'], idf['Longitude'], idf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_indore)  
    
map_indore

In [16]:
#save map as html file
map_indore.save('map_indore.html')

## **5. Use the Foursquare API to explore the neighborhoods**

In [17]:
CLIENT_ID = 'OXJXVBTIJOUXCAEN3W4IYSCFSNC2EBWEXXAEEV1UWEWHYTGM' 
CLIENT_SECRET = 'WRCMCR230EYROYJ0VL4HZCAMMMEHMCGW42PNDBUUORDWUXXK'
VERSION = '20180605'

In [74]:
radius = 15000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(idf['Latitude'], idf['Longitude'], idf['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [75]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VLatitude', 'VLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(1214, 7)


,Neighborhood,Latitude,Longitude,VenueName,VLatitude,VLongitude,VenueCategory
0,"Azad Nagar, Indore",22.69658,75.88767,56 Dukan,22.724118,75.884810,Plaza
1,"Azad Nagar, Indore",22.69658,75.88767,PVR Cinemas,22.721111,75.878536,Movie Theater
2,"Azad Nagar, Indore",22.69658,75.88767,Shreemaya Hotel Indore,22.715057,75.874613,Hotel
3,"Azad Nagar, Indore",22.69658,75.88767,Naturals,22.718133,75.872239,Ice Cream Shop
4,"Azad Nagar, Indore",22.69658,75.88767,Mr Beans,22.723269,75.896949,Cafeteria
5,"Azad Nagar, Indore",22.69658,75.88767,Cafe Palette,22.721373,75.887460,Café
6,"Azad Nagar, Indore",22.69658,75.88767,Chopstick City,22.751427,75.889599,Chinese Restaurant
7,"Azad Nagar, Indore",22.69658,75.88767,Sayaji Hotel,22.751315,75.889490,Hotel
8,"Azad Nagar, Indore",22.69658,75.88767,Sky Sheesha Lounge,22.744123,75.894624,Hookah Bar
9,"Azad Nagar, Indore",22.69658,75.88767,Mediterra,22.751561,75.889079,Mediterranean Restaurant


In [76]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VLatitude,VLongitude,VenueCategory
Neighborhood,,,,,,
"Azad Nagar, Indore",69,69,69,69,69,69
Bhanwar Kua,69,69,69,69,69,69
Bijalpur,69,69,69,69,69,69
"Chandan Nagar, Indore",69,69,69,69,69,69
"Chandra Nagar, Indore",71,71,71,71,71,71
Dewas Naka,71,71,71,71,71,71
Geeta Bhawan,76,76,76,76,76,76
Khandwa Naka,69,69,69,69,69,69
L.I.G. Colony,72,72,72,72,72,72


In [77]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 35 uniques categories.


In [78]:
 venues_df['VenueCategory'].unique()

array(['Plaza', 'Movie Theater', 'Hotel', 'Ice Cream Shop', 'Cafeteria',
       'Café', 'Chinese Restaurant', 'Hookah Bar',
       'Mediterranean Restaurant', 'Fast Food Restaurant', 'Coffee Shop',
       'Indian Restaurant', 'Sandwich Place', 'Shopping Mall', 'Diner',
       'Multiplex', 'Snack Place', 'Pub', 'Restaurant', 'Park',
       'Pizza Place', 'Dessert Shop', 'Lounge', 'Historic Site',
       'Tea Room', 'Hot Dog Joint', 'Other Great Outdoors', 'BBQ Joint',
       'Bakery', 'Gym', 'Asian Restaurant', 'Airport', 'Airport Terminal',
       'Airport Food Court', 'Resort'], dtype=object)

In [79]:
#"Neighborhood" in venues_df['VenueCategory'].unique()

## **6. Analyze Each Neighborhood**

In [80]:
ionehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ionehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ionehot.columns[-1]] + list(ionehot.columns[:-1])
ionehot = ionehot[fixed_columns]

print(ionehot.shape)
ionehot.head()

(1214, 36)


,Neighborhood,Airport,Airport Food Court,Airport Terminal,Asian Restaurant,BBQ Joint,Bakery,Cafeteria,Café,Chinese Restaurant,...,Park,Pizza Place,Plaza,Pub,Resort,Restaurant,Sandwich Place,Shopping Mall,Snack Place,Tea Room
0,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Azad Nagar, Indore",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
#combining all type of restaurants
ionehot['rest']=ionehot['Mediterranean Restaurant']+ionehot['Restaurant']+ionehot['Indian Restaurant']+ionehot['Chinese Restaurant']+ionehot['Fast Food Restaurant']
#+ionehot['Italian Restaurant']
#+ionehot['Greek Restaurant']
ionehot

,Neighborhood,Airport,Airport Food Court,Airport Terminal,Asian Restaurant,BBQ Joint,Bakery,Cafeteria,Café,Chinese Restaurant,...,Pizza Place,Plaza,Pub,Resort,Restaurant,Sandwich Place,Shopping Mall,Snack Place,Tea Room,rest
0,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Azad Nagar, Indore",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Azad Nagar, Indore",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
7,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Azad Nagar, Indore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### **Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [83]:
i_grouped = ionehot.groupby(["Neighborhood"]).mean().reset_index()

print(i_grouped.shape)
i_grouped

(17, 37)


,Neighborhood,Airport,Airport Food Court,Airport Terminal,Asian Restaurant,BBQ Joint,Bakery,Cafeteria,Café,Chinese Restaurant,...,Pizza Place,Plaza,Pub,Resort,Restaurant,Sandwich Place,Shopping Mall,Snack Place,Tea Room,rest
0,"Azad Nagar, Indore",0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.014493,0.043478,0.014493,...,0.057971,0.014493,0.014493,0.000000,0.043478,0.043478,0.028986,0.028986,0.028986,0.275362
1,Bhanwar Kua,0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.014493,0.043478,0.014493,...,0.057971,0.014493,0.014493,0.000000,0.043478,0.043478,0.028986,0.028986,0.028986,0.275362
2,Bijalpur,0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.014493,0.043478,0.014493,...,0.057971,0.014493,0.014493,0.000000,0.043478,0.043478,0.028986,0.028986,0.028986,0.275362
3,"Chandan Nagar, Indore",0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.014493,0.043478,0.014493,...,0.057971,0.014493,0.014493,0.000000,0.043478,0.043478,0.028986,0.028986,0.028986,0.275362
4,"Chandra Nagar, Indore",0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.042254,0.014085,...,0.070423,0.014085,0.014085,0.000000,0.028169,0.042254,0.028169,0.028169,0.028169,0.253521
5,Dewas Naka,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.042254,0.014085,...,0.070423,0.014085,0.014085,0.000000,0.028169,0.042254,0.028169,0.028169,0.028169,0.253521
6,Geeta Bhawan,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.039474,0.013158,...,0.065789,0.013158,0.013158,0.013158,0.039474,0.039474,0.026316,0.026316,0.026316,0.276316
7,Khandwa Naka,0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.014493,0.043478,0.014493,...,0.057971,0.014493,0.014493,0.000000,0.043478,0.043478,0.028986,0.028986,0.028986,0.275362
8,L.I.G. Colony,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.041667,0.013889,...,0.069444,0.013889,0.013889,0.013889,0.027778,0.041667,0.027778,0.027778,0.027778,0.250000
9,"Malviya Nagar, Indore",0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.042254,0.014085,...,0.070423,0.014085,0.014085,0.000000,0.028169,0.042254,0.028169,0.028169,0.028169,0.253521


In [84]:
len(i_grouped[i_grouped["rest"] > 0])

17

**Create a new DataFrame for Restaurant data only**

In [85]:
i_rest = i_grouped[["Neighborhood","rest"]]
i_rest

,Neighborhood,rest
0,"Azad Nagar, Indore",0.275362
1,Bhanwar Kua,0.275362
2,Bijalpur,0.275362
3,"Chandan Nagar, Indore",0.275362
4,"Chandra Nagar, Indore",0.253521
5,Dewas Naka,0.253521
6,Geeta Bhawan,0.276316
7,Khandwa Naka,0.275362
8,L.I.G. Colony,0.250000
9,"Malviya Nagar, Indore",0.253521


## **7. Cluster Neighborhoods**

In [86]:
kclusters = 3

i_clustering = i_rest.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(i_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 2, 0, 0, 2, 2], dtype=int32)

In [87]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
i_merged = i_rest.copy()

# add clustering labels
i_merged["Cluster Labels"] = kmeans.labels_

In [88]:
i_merged.rename(columns={"rest": "Restaurant"}, inplace=True)
i_merged

,Neighborhood,Restaurant,Cluster Labels
0,"Azad Nagar, Indore",0.275362,0
1,Bhanwar Kua,0.275362,0
2,Bijalpur,0.275362,0
3,"Chandan Nagar, Indore",0.275362,0
4,"Chandra Nagar, Indore",0.253521,2
5,Dewas Naka,0.253521,2
6,Geeta Bhawan,0.276316,0
7,Khandwa Naka,0.275362,0
8,L.I.G. Colony,0.250000,2
9,"Malviya Nagar, Indore",0.253521,2


In [89]:
i_merged = i_merged.join(idf.set_index("Neighborhood"), on="Neighborhood")

print(i_merged.shape)
i_merged

(17, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,"Azad Nagar, Indore",0.275362,0,22.696580,75.887670
1,Bhanwar Kua,0.275362,0,22.687640,75.862910
2,Bijalpur,0.275362,0,22.663240,75.843320
3,"Chandan Nagar, Indore",0.275362,0,22.709380,75.823940
4,"Chandra Nagar, Indore",0.253521,2,22.741470,75.894470
5,Dewas Naka,0.253521,2,22.771764,75.901659
6,Geeta Bhawan,0.276316,0,22.718220,75.887660
7,Khandwa Naka,0.275362,0,22.689258,75.870259
8,L.I.G. Colony,0.250000,2,22.734780,75.884850
9,"Malviya Nagar, Indore",0.253521,2,22.741695,75.899889


In [90]:
print(i_merged.shape)
i_merged.sort_values(["Cluster Labels"], inplace=True)
i_merged

(17, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,"Azad Nagar, Indore",0.275362,0,22.696580,75.887670
1,Bhanwar Kua,0.275362,0,22.687640,75.862910
2,Bijalpur,0.275362,0,22.663240,75.843320
3,"Chandan Nagar, Indore",0.275362,0,22.709380,75.823940
6,Geeta Bhawan,0.276316,0,22.718220,75.887660
7,Khandwa Naka,0.275362,0,22.689258,75.870259
12,"Rajendra Nagar, Indore",0.275362,0,22.674160,75.842240
10,Navlakha,0.276316,0,22.704790,75.876570
11,Palasia,0.276316,0,22.727690,75.885780
15,"Sukhliya, Indore",0.231884,1,22.760220,75.878150


**Finally, let's visualize the resulting clusters**

In [91]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(i_merged['Latitude'], i_merged['Longitude'], i_merged['Neighborhood'], i_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [92]:
#save map as html file
map_clusters.save('map_clusters.html')

## **8. Examine Clusters**

### **A. cluster 0**

In [93]:
i_merged.loc[i_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,"Azad Nagar, Indore",0.275362,0,22.696580,75.887670
1,Bhanwar Kua,0.275362,0,22.687640,75.862910
2,Bijalpur,0.275362,0,22.663240,75.843320
3,"Chandan Nagar, Indore",0.275362,0,22.709380,75.823940
6,Geeta Bhawan,0.276316,0,22.718220,75.887660
7,Khandwa Naka,0.275362,0,22.689258,75.870259
12,"Rajendra Nagar, Indore",0.275362,0,22.674160,75.842240
10,Navlakha,0.276316,0,22.704790,75.876570
11,Palasia,0.276316,0,22.727690,75.885780


### **B. Cluster 1**

In [94]:
i_merged.loc[i_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
15,"Sukhliya, Indore",0.231884,1,22.76022,75.87815


### **C. Cluster 2**

In [95]:
i_merged.loc[i_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
14,Satya Sai Chouraha,0.260274,2,22.755351,75.896877
13,Rau Colony,0.250000,2,22.734780,75.884850
8,L.I.G. Colony,0.250000,2,22.734780,75.884850
5,Dewas Naka,0.253521,2,22.771764,75.901659
4,"Chandra Nagar, Indore",0.253521,2,22.741470,75.894470
9,"Malviya Nagar, Indore",0.253521,2,22.741695,75.899889
16,"Vijay Nagar, Indore",0.260274,2,22.756410,75.892470


## **Observations:**
1. cluster 2 show higgest number of restaurant.
2. cluster 0 show average number of restaurant available in most areas.
3. cluster 1 show there is less number of restaurant available in that area.  
**From the observation while thinking of setting up a restaurant in indore it is more beneficial to select a place where there is least number of restaurants so that people there get chances to have something close to their place.
Therefore, this project recommends property developers or restaurants owner to capitalize on these findings to open new Restaurants in neighborhoods in cluster 1 with little to no competition.
Property developers with unique selling propositions to stand out from the competition can also open new Restaurant in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of restaurants and suffering from intense competition.**